In [1]:
class EnviromentReport():
    
    def __init__(self):
        self.energy_consumption_entries = []
        
    def add_entry(self, amount_energy, total_time, vessel):
        self.energy_consumption_entries.append((amount_energy, total_time, vessel))
        
    def get_energy_usage(self):
        filtered_result = {}
        for energy_entry in self.energy_consumption_entries:
            vessel = energy_entry[2]
            path_name = vessel.path_complete.name
            
            if path_name in filtered_result:
                if (vessel.loaded):
                    filtered_result[path_name]["loaded"] += energy_entry[0] * 10**-3 #van kwh naar MWh
                else:
                    filtered_result[path_name]["unloaded"] += energy_entry[0] * 10**-3
            else:
                if (vessel.loaded):
                    filtered_result[path_name] = {"loaded": energy_entry[0] * 10**-3, "unloaded": 0}
                else:
                    filtered_result[path_name] = {"loaded": 0, "unloaded": energy_entry[0] * 10**-3}

        return filtered_result
    
    def get_co2_emission_load_km(self):
        filtered_result = {}
        moved_load_km = {}
        for energy_entry in self.energy_consumption_entries:
            vessel = energy_entry[2]
            path_name = vessel.path_complete.name
            
            if path_name in moved_load_km and vessel.loaded:
                moved_load_km[path_name] += vessel.distance * 10**-3  * vessel.loaded_capacity

            if (path_name not in moved_load_km and vessel.loaded):
                moved_load_km[path_name] = vessel.distance * 10**-3 * vessel.loaded_capacity
            
            if path_name in filtered_result:
                    filtered_result[path_name] += energy_entry[0]  * vessel.emission *10**3 

            else:
                    filtered_result[path_name] = energy_entry[0] * vessel.emission * 10**3
        
        for key in moved_load_km.keys():
            filtered_result[key] = filtered_result[key] / (moved_load_km[key])
        return filtered_result
    
    
    def get_co2_emission(self):
        filtered_result = {}
        for energy_entry in self.energy_consumption_entries:
            vessel = energy_entry[2]
            path_name = vessel.path_complete.name
            
            if path_name in filtered_result:
                if (vessel.loaded):
                    filtered_result[path_name]["loaded"] += energy_entry[0]  * vessel.emission * 10**-3  # 10^3 to get from kg to ton
                else:
                    filtered_result[path_name]["unloaded"] += energy_entry[0]  * vessel.emission * 10**-3
            else:
                if (vessel.loaded):
                    filtered_result[path_name] = {"loaded": energy_entry[0] * vessel.emission * 10**-3, "unloaded": 0}
                else:
                    filtered_result[path_name] = {"loaded": 0, "unloaded": energy_entry[0] * vessel.emission * 10**-3}

        return filtered_result
    
    
    def get_transported_weight_corridor(self):
        filtered_result = {}
        for energy_entry in self.energy_consumption_entries:
            vessel = energy_entry[2]
            path_name = vessel.path_complete.name
            
            if path_name in filtered_result:
                if (vessel.loaded):
                    filtered_result[path_name]["loaded"] += vessel.loaded_capacity
                else:
                    filtered_result[path_name]["unloaded"] += 0
            else:
                if (vessel.loaded):
                    filtered_result[path_name] = {"loaded": vessel.loaded_capacity, "unloaded": 0}
                else:
                    filtered_result[path_name] = {"loaded": 0, "unloaded": vessel.loaded_capacity}

        return filtered_result
    
    
    def plot_transported_weight_corridor(self, save=False):
        plot_dict = self.get_transported_weight_corridor()
        
        loaded = [i["loaded"] for i in plot_dict.values()]
        unloaded = [i["unloaded"] for i in plot_dict.values()]
        loaded.append(np.sum(loaded))
        unloaded.append(np.sum(unloaded))
        plot_dict["Total"] = {}
        width = 0.35       # the width of the bars: can also be len(x) sequence
        plt.figure(figsize=(6,6))
        plt_unloaded = plt.bar(range(len(plot_dict)), unloaded, width, bottom=loaded, align='center')
        plt_loaded = plt.bar(range(len(plot_dict)), loaded, width, align='center')
        
        plt.xticks(range(len(plot_dict)), list(plot_dict.keys()))
        plt.legend((plt_unloaded[0], plt_loaded[0]), ('unloaded vessels', 'Loaded vessels'))
        
        plt.ylabel('Ton')
        plt.title('Transported weight over {} days categorized by paths'\
                  .format(int(self.energy_consumption_entries[-1][2].env.now / 86400)))
        #plt.title('{} vessels'.format(len(self.energy_consumption_entries)))
        #if save:
        plt.savefig('Resultaten/Transported_weight' + str(i) +'.png')
        plt.show()
    
    
    def plot_co2_emission(self, save=False):
        plot_dict = self.get_co2_emission()
        
        loaded = [i["loaded"] for i in plot_dict.values()]
        unloaded = [i["unloaded"] for i in plot_dict.values()]
        loaded.append(np.sum(loaded))
        unloaded.append(np.sum(unloaded))
        plot_dict["Total"] = {}
        width = 0.35       # the width of the bars: can also be len(x) sequence
        plt.figure(figsize=(6,6))
        plt_unloaded = plt.bar(range(len(plot_dict)), unloaded, width, bottom=loaded, align='center')
        plt_loaded = plt.bar(range(len(plot_dict)), loaded, width, align='center')
        
        plt.xticks(range(len(plot_dict)), list(plot_dict.keys()))
        plt.legend((plt_unloaded[0], plt_loaded[0]), ('unloaded vessels', 'Loaded vessels'))
        
        plt.ylabel('Ton')
        plt.title('Total fleet CO2 emission over {} days categorized by paths'\
                  .format(int(self.energy_consumption_entries[-1][2].env.now / 86400)))
        #plt.title('{} vessels'.format(len(self.energy_consumption_entries)))
        #if save:
        plt.savefig('Resultaten/CO2_emission_ton' + str(i) +'.png')
        plt.show()
        
    def plot_co2_emission_km_load(self, save=False):
        plot_dict =  self.get_co2_emission_load_km()
        #plot_dict["total"] = np.sum(list(plot_dict.values()))
        width = 0.35       # the width of the bars: can also be len(x) sequence
        plt.figure(figsize=(5,5))
        
        plt.bar(range(len(plot_dict)), list(plot_dict.values()), width, align='center')
        
        plt.xticks(range(len(plot_dict)), list(plot_dict.keys()))

        
        plt.ylabel('g/Ton KM')
        plt.title('Total fleet CO2/Ton Km emission over {} days categorized by paths'\
                  .format(int(self.energy_consumption_entries[-1][2].env.now / 86400)))
        #plt.title('{} vessels'.format(len(self.energy_consumption_entries)))
        #if save:
        plt.savefig('Resultaten/co2_emission_km_load' + str(i) +'.png')
        plt.show()
        
    
    def plot_energy_usage(self, save=False):
        plot_dict = self.get_energy_usage()
        
        loaded = [i["loaded"] for i in plot_dict.values()]
        unloaded = [i["unloaded"] for i in plot_dict.values()]
        loaded.append(np.sum(loaded))
        unloaded.append(np.sum(unloaded))
        plot_dict["Total"] = {}
        width = 0.35       # the width of the bars: can also be len(x) sequence
        plt.figure(figsize=(6,6))
        plt_unloaded = plt.bar(range(len(plot_dict)), unloaded, width, bottom=loaded, align='center')
        plt_loaded = plt.bar(range(len(plot_dict)), loaded, width, align='center')
        
        plt.xticks(range(len(plot_dict)), list(plot_dict.keys()))
        plt.legend((plt_unloaded[0], plt_loaded[0]), ('unloaded vessels', 'Loaded vessels'))
        
        plt.ylabel('MWh')
        plt.title('Total fleet energy usage over {} days categorized by paths'\
                  .format(int(self.energy_consumption_entries[-1][2].env.now / 86400)))
        #plt.title('{} vessels'.format(len(self.energy_consumption_entries)))
        #if save:
        plt.savefig('Resultaten/energy_usage_MWh' + str(i) +'.png')
        plt.show()
        
        
        
    def plot_path_usage(self):
        plt.figure(figsize=(6,6))
        plt.title("Vessel distribution over paths")
        plt.ylabel('Number of vessels')
        plot_dict = self.energy_consumption_entries[-1][2].env.paths


        plot_dict["Total"] = np.sum(list(plot_dict.values()))
        plt.bar(range(len(plot_dict)), list(plot_dict.values()), align='center')
        plt.xticks(range(len(plot_dict)), list(plot_dict.keys()))
        plt.savefig('Resultaten/path_usage' + str(i) +'.png')
        plt.show()

    def report(self):
        print(self.energy_consumption_entries)
        